In [1]:
import pandas as pd
import bq_helper
from bq_helper import BigQueryHelper
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from guesslang import Guess
import os
import pickle
import warnings

warnings.filterwarnings('ignore')

# Read the two datasets containing repositories and merge in one dataset

In [10]:
repositories_c = pd.read_csv("Repositories\\repositories_c.csv",usecols=[0])
repositories_c.head()

,name
0,google/brotli
1,gentilkiwi/mimikatz
2,octalmage/robotjs
3,openresty/openresty
4,jedisct1/libsodium


In [11]:
repositories_cpp = pd.read_csv("Repositories\\repositories_cpp.csv",usecols=[0])
repositories_cpp.head()

,name
0,google/tink
1,lottie-react-native/lottie-react-native
2,tencent/matrix
3,topjohnwu/magisk
4,microsoft/cntk


In [13]:
repositories_final = pd.concat([repositories_c, repositories_cpp]).drop_duplicates().reset_index(drop=True)
repositories_final

,name
0,google/brotli
1,gentilkiwi/mimikatz
2,octalmage/robotjs
3,openresty/openresty
4,jedisct1/libsodium
...,...
128,notepad-plus-plus/notepad-plus-plus
129,apple/swift
130,pytorch/pytorch
131,onnx/onnx


In [15]:
repositories_final.to_csv("Repositories/repositories_final.csv", index=False)

# Read posts from Stack Overflow with code snippets in the period from 2015 to 2020

In [19]:
sotorrent = bq_helper.BigQueryHelper("sotorrent-org", "2020_12_31")

In [21]:
query = "SELECT id, body FROM `sotorrent-org.2020_12_31.Posts` WHERE  body LIKE '%<code>%</code>%' AND (LastEditDate>='2015-01-01T00:00:00' AND LastEditDate<='2020-12-31T23:59:59')"
post_with_code_snippets = sotorrent.query_to_pandas_safe(query, max_gb_scanned = 70)
post_with_code_snippets

,id,body
0,58436889,<p>Use <code>zip()</code> with a dictionary-co...
1,45947811,"<p>You can look at <a href=""https://github.com..."
2,45452106,<p>You need to handle the check state of the c...
3,46393236,<p>My Goal:<br>\n I want to search for ...
4,55404818,"<blockquote>\n <p>Here, it says 'identifier ""..."
...,...,...
11929628,35781386,<h2>XSD 1.0</h2>\n\n<p>Along the lines <a href...
11929629,57029348,"<p>I already asked about the problem, but then..."
11929630,59489412,"<p>I am new to python and this code is like ,w..."
11929631,57725752,<p>This will Works by Change if Condition.</p>...


In [23]:
post_with_code_snippets.to_csv("Datasets Code Snippets/post_with_code_snippets.csv.gz", 
           index=False, 
           compression="gzip")

# Extract code content in post body

In [2]:
chunksize = 400000
data_path = "Datasets Code Snippets/post_with_code_snippets.csv.gz"
df_list = [] 

for df_chunk in tqdm(pd.read_csv(data_path, chunksize=chunksize,compression='gzip')):
    df_list.append(df_chunk)

In [15]:
post_with_only_code_snippets = dict()
guess = Guess()
i = 29 #change this variable to execute on differents chunks (0...29)
df_list[i] = df_list[i].reset_index(drop=True)
for index in tqdm(range(df_list[i].shape[0])):
    soup = BeautifulSoup(df_list[i]['body'][index])

    for code in soup.find_all('code'):
        name = guess.language_name(code.text)
        
        if(name=="C" or name=="C++"):
            key = post_with_only_code_snippets.get(df_list[i]['id'][index], None)
            if key:
                post_with_only_code_snippets[df_list[i]['id'][index]].append(code.text)
            else:
                post_with_only_code_snippets[df_list[i]['id'][index]] = list()
                post_with_only_code_snippets[df_list[i]['id'][index]].append(code.text)
            
print(len(post_with_only_code_snippets))
with open('tmp/post_with_only_code_snippets_'+str(i)+".pkl", "wb") as f:
    pickle.dump(post_with_only_code_snippets, f)

Empty source code provided
Empty source code provided
Empty source code provided
Empty source code provided
No programming language detected
No programming language detected
Empty source code provided
Empty source code provided
No programming language detected
No programming language detected
No programming language detected
No programming language detected
No programming language detected
Empty source code provided
Empty source code provided
Empty source code provided
No programming language detected
No programming language detected
Empty source code provided
Empty source code provided
Empty source code provided
Empty source code provided
Empty source code provided
Empty source code provided
Empty source code provided
Empty source code provided
No programming language detected
No programming language detected
Empty source code provided
Empty source code provided
Empty source code provided
No programming language detected
No programming language detected
No programming language detecte

Empty source code provided
Empty source code provided
Empty source code provided
No programming language detected
No programming language detected
No programming language detected
No programming language detected
No programming language detected
No programming language detected
No programming language detected
Empty source code provided
No programming language detected
Empty source code provided
No programming language detected
No programming language detected
No programming language detected
No programming language detected
Empty source code provided
No programming language detected
No programming language detected
Empty source code provided
No programming language detected
Empty source code provided
No programming language detected
No programming language detected
Empty source code provided
No programming language detected
No programming language detected
Empty source code provided
No programming language detected
Empty source code provided
No programming language detected
No program


8556


In [2]:
files = os.listdir('tmp/')
post_with_only_code_snippets = dict()

for file in files:
    with open("tmp/"+file, "rb") as f:
        file_content = pickle.load(f)
        post_with_only_code_snippets = {**post_with_only_code_snippets, **file_content}
        
len(post_with_only_code_snippets)

358230

In [10]:
post_with_only_code_snippets_final = dict()
idx = 0
keys = post_with_only_code_snippets.keys()

for key in tqdm(keys):
    codes = post_with_only_code_snippets[key]
    for code in codes:
        post_with_only_code_snippets_final[idx]= {"id": key, "code": code}
        idx = idx+1
        
        

In [13]:
post_with_only_code_snippets_final = pd.DataFrame.from_dict(post_with_only_code_snippets_final, orient='index')

In [17]:
post_with_only_code_snippets_final

,id,code
0,37637877,void closeBuySell(int buyTicket){\n double b...
1,57596800,"#include <stdio.h>\n\nvoid sum(int *, int *, i..."
2,22667586,"#include ""buraco/game/board/model/KingChatFilt..."
3,22667586,"#include ""buraco\game/board/model\CardGroup.h""..."
4,55014685,#include <stdlib.h> /* EXIT malloc free */\n#i...
...,...,...
452666,64735134,"template <class T, class ...Ts>\nvoid printA(T..."
452667,65106538,#include <iostream>\nusing namespace std;\n\nv...
452668,25117355,#include <iostream>\nusing namespace std;\n\ni...
452669,55782153,#include<stdio.h>\n#include <limits.h>\n#inclu...


In [18]:
post_with_only_code_snippets_final.to_csv("Datasets Code Snippets/post_with_only_code_snippets_final.csv", 
           index=False)